In [10]:
import datetime

import numpy as np
import pandas as pd
import yaml

In [11]:
from sqlalchemy import create_engine

with open('./config.yml', 'r') as f:
        config = yaml.safe_load(f)
        config_ryf = config['RAPIDO_Y_FURIOSO']
        config_etl = config['ETL']

url_ryf = (f"{config_ryf['drivername']}://{config_ryf['user']}:{config_ryf['password']}@{config_ryf['host']}:"
        f"{config_ryf['port']}/{config_ryf['dbname']}")
url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@{config_etl['host']}:"
        f"{config_etl['port']}/{config_etl['dbname']}")

ryf = create_engine(url_ryf)
etl_conn = create_engine(url_etl)

In [12]:
trans_novedad = pd.read_sql_table('mensajeria_novedadesservicio', ryf)
df_tipos_novedad = pd.read_sql_table('mensajeria_tiponovedad', ryf)

In [13]:
trans_novedad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5208 entries, 0 to 5207
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   id               5208 non-null   int64              
 1   fecha_novedad    5208 non-null   datetime64[ns, UTC]
 2   tipo_novedad_id  5208 non-null   int64              
 3   descripcion      5208 non-null   object             
 4   servicio_id      5208 non-null   int64              
 5   es_prueba        5208 non-null   bool               
 6   mensajero_id     5208 non-null   int64              
dtypes: bool(1), datetime64[ns, UTC](1), int64(4), object(1)
memory usage: 249.3+ KB


In [14]:
trans_novedad = trans_novedad.merge(
    df_tipos_novedad, 
    left_on='tipo_novedad_id',
    right_on='id',
    how='left'
)

In [15]:
trans_novedad = trans_novedad.filter(['fecha_novedad', 'tipo_novedad_id', 'descripcion'])

In [18]:
trans_novedad['saved'] = datetime.date.today()
trans_novedad.head()

,fecha_novedad,tipo_novedad_id,descripcion,saved
0,2023-11-30 05:00:00+00:00,1,A,2024-11-10
1,2023-11-30 05:00:00+00:00,1,Halo,2024-11-10
2,2023-11-30 05:00:00+00:00,1,A,2024-11-10
3,2023-11-30 05:00:00+00:00,1,B,2024-11-10
4,2023-11-30 05:00:00+00:00,1,A,2024-11-10


In [19]:
trans_novedad.to_sql('trans_novedad', etl_conn, if_exists='replace', index_label='key_trans_novedad')

208